<a href="https://colab.research.google.com/github/ola808/NLP/blob/master/lab2/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Task 2 (deadline 26.02.2020 24:00)
* It would be best if you created a pandas DataFrame of the texts from the previous task. And apply to the DataFrame the function 'get_hyperonyms,' which must return the list of the corresponding hypernyms from the text automatically. If there are errors or misses, you should fix them in the code for your case of the 24 words. Nevertheless, it is strictly prohibited to use hard coding. Save your notebook with parser code within the lab2 folder in your NLP git-repo.

### Pattern extractor

Yargy — библиотека для извлечения структурированной информации из текстов на русском языке. Правила описываются контекстно-свободными грамматиками и словарями ключевых слов. Банк готовых правил для имён, дат, адресов и других сущностей доступен в репозитории Natasha.
* https://yargy.readthedocs.io/ru/latest/
* http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
* https://github.com/natasha/natasha

In [0]:
!pip install yargy
!pip install wikipedia

     |████████████████████████████████| 51kB 4.2MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 7.1MB 46.7MB/s 
  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=19b67101a5624a0279f93c66714878b4e938ae8f742c7d597231b24e98be74ed
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [0]:
import wikipedia
wikipedia.set_lang("ru")

from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

import warnings
warnings.filterwarnings('ignore')

START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [0]:
def get_hyperonyms(main_word):
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM)
    )
    parser = Parser(RULE) 
    text = utils.deaccent(wikipedia.summary(main_word))
    print(text)
    text = re.sub(r'\(.+?\)', '', text)
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        print(k)

In [0]:
import pandas as pd

CHARGE_NUMBER = 54

df = pd.read_csv("Lab2_words.csv")
charge = df.loc[df['IN CHARGE NUMBER'] == CHARGE_NUMBER]
charge.head(5)

,Заполнено,Unnamed: 1,IN CHARGE NUMBER,IN CHARGE NAME,HYPONYM,HYPERNYM,HYPERNYM.1,HYPERNYM.2,HYPERNYM.3,HYPERNYM.4,HYPERNYM.5,HYPERNYM.6,HYPERNYM.7,HYPERNYM.8,HYPERNYM.9,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,#ERROR!
1272,NaN,1273,54,Olzhas Amirov,ТАКСИДЕРМИЯ,болезнь,профессия,хобби,наука,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1273,NaN,1274,54,Olzhas Amirov,ТАЛЛИЙ,элемент,болезнь,вещество,яд,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1274,NaN,1275,54,Olzhas Amirov,ТАНИН,порошок,вещество,кислота,лекарство,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1275,NaN,1276,54,Olzhas Amirov,ТАНКЕТКА,обувь,машина,стиль,ведомство,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1276,NaN,1277,54,Olzhas Amirov,ТАПИР,животные,млекопитающие,лошадь,фауна,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
for index, row in charge.iterrows():
    try:
      print(row['HYPONYM'])
      get_hyperonyms(row['HYPONYM'])
      print("-"*50)
    except: 
        pass

ТАКСИДЕРМИЯ
Таксидермия (др.-греч. ταξις — приведение в порядок и δερμα — кожа, шкура) — способ изготовления реалистичных изображении (чучел) животных, основои при котором является шкура животного. Она собирается на некую основу, а полость внутри заполняется наполнителем. Современная таксидермия использует «манекены» (внутреннюю основу из пенополиуретана с анатомическим сохранением пропорции всех частеи тела и объема мышц). В основном таксидермия применяется для изготовления музеиных экспонатов, а также для сохранения охотничьих трофеев.
--------------------------------------------------
ТАЛЛИЙ
Таллии — элемент 13-и группы периодическои таблицы химических элементов (по устаревшеи классификации — элемент главнои подгруппы III группы), шестого периода, атомныи номер 81. Обозначается символом Tl (лат. Thallium). Относится к группе тяжелых металлов. Простое вещество таллии — мягкии, чрезвычаино токсичныи металл серебристо-белого цвета с серовато-голубоватым оттенком.
----------------------